<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/finetune_alephbert_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Aug 19 19:01:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Connect to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !gzip -d /content/drive/MyDrive/nlp/squad2.0/train_preprocessed_translated_squad.csv.gz
# !gzip -d /content/drive/MyDrive/nlp/squad2.0/val_preprocessed_translated_squad.tsv

In [5]:
pip install transformers

     |████████████████████████████████| 2.6 MB 8.6 MB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 3.3 MB 64.5 MB/s 
     |████████████████████████████████| 636 kB 67.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import collections
import pandas as pd
from tqdm.notebook import trange, tqdm

import torch
from torch.utils.data import DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
BATCH_SIZE = 16

In [8]:
df = pd.read_csv('/content/drive/MyDrive/nlp/squad2.0/train_preprocessed_translated_squad.csv')
val_df = pd.read_csv('/content/drive/MyDrive/nlp/squad2.0/val_preprocessed_translated_squad.tsv')

In [9]:
df = df[(df.answer_start_heb > 0) & (df.answer_end_heb > 0)]
val_df = val_df[(val_df.answer_start_heb > 0) & (val_df.answer_end_heb > 0)]

## Transform the texts to encodings

In [10]:
train_ids = df.id.to_list()
train_contexts = df.context.to_list()
train_questions = df.question.to_list()

val_ids = val_df.id.to_list()
val_contexts = val_df.context.to_list()
val_questions = val_df.question.to_list()

In [11]:
train_answers = df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()
val_answers = val_df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()

In [12]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/545k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

In [13]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [14]:
train_encodings.update({'id': train_ids})
val_encodings.update({'id': val_ids})

## Create PyTorch dataset

In [15]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Get all encoded vals as tensors
        vals = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key != 'id'}
        # Add the id which is a string, used to map predictions to ids later
        vals.update({key: val[idx] for key, val in self.encodings.items() if key == 'id'})
        return vals

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Import Hebrew BERT for Q&A fine tuning

In [17]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained("onlplab/alephbert-base")

Downloading:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initi

In [18]:
# PATH = '/content/drive/MyDrive/nlp/checkpoints/finetuned_alephbert/checkpoint_0'

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# model.eval()
# - or -
# model.train()

## Code to evaluate model

In [19]:
def calc_metrics(model_inf, dataloader):
  """
  Given a model and a dataset calculate the following metrics:
  - Exact Match
  - F1 Score
  - Loss
  @returns (F1, EM, Loss, EM Scores, F1 Scores)
  """

  model.to(device)
  model.eval()

  f1_scores = {}
  exact_scores = {}

  loss = 0
  total_cnt = 0

  for counter, batch in tqdm(enumerate(dataloader)):
    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      ids = batch['id']
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss += outputs[0]

      pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
      pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

      # Calc scores
      for i in range(len(input_ids)):

        total_cnt += 1

        # Check if we have an exact match
        if start_positions[i] == pred_start_positions[i] and end_positions[i] == pred_end_positions[i]:
          # We have an exact match, mark it
          exact_scores[ids[i]] = 1
        else:
          # No match, mark with 0 or keep previous marking
          exact_scores[ids[i]] = max(0, exact_scores.get(ids[i], 0))

        # Get the predicted answer token sequence
        pred_tokens = input_ids[i][pred_start_positions[i]:pred_end_positions[i] + 1].tolist()
        answer_tokens = input_ids[i][start_positions[i]:end_positions[i] + 1].tolist()

        # Check how many predicted tokens correspond to gold answer tokens
        common = collections.Counter(answer_tokens) & collections.Counter(pred_tokens)
        num_same = sum(common.values())

        # If none same, then F1=0 , else use the F1 formula
        if num_same == 0:
          score = 0
        else:
          precision = 1.0 * num_same / len(pred_tokens)
          recall = 1.0 * num_same / len(answer_tokens)
          score = (2 * precision * recall) / (precision + recall)
        
        # We take the max F1 score of the gold answers
        f1_scores[ids[i]] = max(score, f1_scores.get(ids[i], 0))

  f1 = sum(f1_scores.values()) / len(f1_scores)
  em = sum(exact_scores.values()) / len(exact_scores)
  epoch_avg_loss = loss / total_cnt

  return (f1, em, epoch_avg_loss, exact_scores, f1_scores)

## Train model

In [ ]:
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

optim = AdamW(model.parameters(), lr=5e-5)

train_loss = []
val_loss = []

for epoch in range(4):

    model.train()

    epoch_loss = 0

    for counter, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        epoch_loss += loss

        if counter % 250 == 0 and counter > 0:
            print('Epoch %d, %d/%d, Current Loss = %.4f' % (epoch, counter, len(train_loader), loss))

    print('Epoch %d, Loss = %.4f' % (epoch, epoch_loss / len(train_loader)))

    f1, em, avg_val_loss, _, _ = calc_metrics(model, val_loader)

    train_loss.append(epoch_loss / len(train_loader))
    val_loss.append(avg_val_loss)

    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optim.state_dict(),
        'f1': f1,
        'em': em,
        'avg_val_loss': avg_val_loss
    }
    savepath=f'/content/drive/MyDrive/nlp/checkpoints/checkpoint_{epoch}_{f1:.3f}_{em:.3f}_{avg_val_loss:.3f}'
    torch.save(state, savepath)

Epoch 0, 250/4565, Current Loss = 2.1486
Epoch 0, 500/4565, Current Loss = 1.6689
Epoch 0, 750/4565, Current Loss = 2.4714
Epoch 0, 1000/4565, Current Loss = 0.9343
Epoch 0, 1250/4565, Current Loss = 1.0518
Epoch 0, 1500/4565, Current Loss = 1.8170
Epoch 0, 1750/4565, Current Loss = 2.0870
Epoch 0, 2000/4565, Current Loss = 0.8047
Epoch 0, 2250/4565, Current Loss = 1.0485
Epoch 0, 2500/4565, Current Loss = 1.2231
Epoch 0, 2750/4565, Current Loss = 1.5483
Epoch 0, 3000/4565, Current Loss = 1.1691
Epoch 0, 3250/4565, Current Loss = 1.3767
Epoch 0, 3500/4565, Current Loss = 1.6407
Epoch 0, 3750/4565, Current Loss = 1.6286
Epoch 0, 4000/4565, Current Loss = 1.4349
Epoch 0, 4250/4565, Current Loss = 1.9330
Epoch 0, 4500/4565, Current Loss = 1.4860
Epoch 0, Loss = 1.7221


0it [00:00, ?it/s]

Epoch 1, 250/4565, Current Loss = 0.6367
Epoch 1, 500/4565, Current Loss = 0.5394
Epoch 1, 750/4565, Current Loss = 1.1141
Epoch 1, 1000/4565, Current Loss = 0.5974
Epoch 1, 1250/4565, Current Loss = 1.7289
Epoch 1, 1500/4565, Current Loss = 1.1822
Epoch 1, 1750/4565, Current Loss = 0.5699
Epoch 1, 2000/4565, Current Loss = 1.1819
Epoch 1, 2250/4565, Current Loss = 2.0615
Epoch 1, 2500/4565, Current Loss = 0.7599
Epoch 1, 2750/4565, Current Loss = 0.5427
Epoch 1, 3000/4565, Current Loss = 1.6593
Epoch 1, 3250/4565, Current Loss = 0.7919
Epoch 1, 3500/4565, Current Loss = 1.7748
Epoch 1, 3750/4565, Current Loss = 0.5353
Epoch 1, 4000/4565, Current Loss = 0.9791
Epoch 1, 4250/4565, Current Loss = 1.1648
Epoch 1, 4500/4565, Current Loss = 1.5581
Epoch 1, Loss = 1.0840


0it [00:00, ?it/s]

Epoch 2, 250/4565, Current Loss = 0.8529


In [ ]:
total_cnt = 0
tp_start = 0
tp_end = 0

for counter, batch in enumerate(train_loader):
  with torch.no_grad():

    total_cnt += input_ids.size()[0]

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    ids = batch['id']
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    
    pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
    pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

    # Array of True/False if pred is same as label
    comp_start = start_positions == pred_start_positions
    comp_end = end_positions == pred_end_positions

    # True = 1 and False = 0 therefore we count how many True
    tp_start += comp_start.count_nonzero()
    tp_end += comp_end.count_nonzero()

print(tp_start, tp_end, total_cnt)
print(tp_start / total_cnt, tp_end / total_cnt)

In [ ]:
# def infer(model_inf, dataloader)

In [ ]:
def map_id_to_gold_answers(df):
  """
  In the SQuaD dev set there are multiple correct (gold) answers.
  This methods creates a mapping between the question id to
  its gold answers
  """
  ids_to_gold = {}
  ids_to_context = {}

  # Group by id as validation set has several gold answer
  for id, cols in df.groupby(by='id'):
    gold_answers = [x[1].context[x[1].answer_start_heb:x[1].answer_end_heb] for x in cols.iterrows()]
    context = cols.iloc[0].context

    # TODO: Normalize answers!

    ids_to_gold[id] = gold_answers
    ids_to_context[id] = context
  
  return ids_to_gold, ids_to_context

val_gold_answers, val_contexts = map_id_to_gold_answers(val_df)

In [ ]:
model.to(device)

predictions = {}

for counter, batch in enumerate(val_loader):
  with torch.no_grad():
  
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    ids = batch['id']
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    
    pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
    pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

    # print(pred_start_positions, pred_end_positions)

    for i in range(1):
      # context = val_contexts.get(ids[i])
      x = tokenizer.convert_ids_to_tokens(input_ids[i])
      # print(x[pred_start_positions[i]:pred_end_positions[i]+1])

      # Start with the first token.
      answer = x[pred_start_positions[i]]

      # Select the remaining answer tokens and join them with whitespace.
      for j in range(pred_start_positions[i] + 1, pred_end_positions[i] + 1):
          
          # If it's a subword token, then recombine it with the previous token.
          if x[j][0:2] == '##':
              answer += x[j][2:]
          
          # Otherwise, add a space then the token.
          else:
              answer += ' ' + x[j]
      predictions[ids[i]] = answer

      # start = val_encodings.token_to_chars(i, pred_start_positions[i])
      # end = val_encodings.token_to_chars(i, pred_end_positions[i])
      # pred_answer = context[start.start:end.end]
      # print(pred_answer)


In [76]:
import collections
import numpy as np

collections.Counter(list(input_ids[0][pred_start_positions[0]:pred_end_positions[0]+1])) & collections.Counter(tokenizer([answer])['input_ids'][0])

Counter()

In [78]:
a = np.array(tokenizer([answer])['input_ids'][0])
b = input_ids[0][pred_start_positions[0]:pred_end_positions[0]+1].cpu().numpy()

In [81]:
a[:4]&b

array([   2, 1028,    4,    2])

In [70]:
collections.Counter(a) & collections.Counter(b)

Counter({6: 1, 178: 1, 1908: 1, 3214: 1})

In [58]:
input_ids[0][pred_start_positions[0]:pred_end_positions[0]+1]

tensor([3214, 1908,    6,  178], device='cuda:0')

In [ ]:
input_ids[0][start_positions[0]:end_positions[0]+1]

tensor([  53, 5543], device='cuda:0')

In [48]:
answer

'כוח הק " ג'

In [55]:
from tqdm.notebook import trange, tqdm

def calc_metrics(model_inf, dataloader):
  """
  Given a model and a dataset calculate the following metrics for the answer
  start and end:
  - Accuracy
  - ?
  @returns (accuracy, F1)
  """

  model.to(device)
  model.eval()

  # tp_start = 0
  # tp_end = 0

  f1_scores = {}
  exact_scores = {}

  loss = 0
  total_cnt = 0

  for counter, batch in tqdm(enumerate(dataloader)):
    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      ids = batch['id']
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss += outputs[0]

      pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
      pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

      # # Array of True/False if pred is same as label
      # comp_start = start_positions == pred_start_positions
      # comp_end = end_positions == pred_end_positions
      # score = sum(comp_start.cpu().numpy() & comp_end.cpu().numpy())
      
      # # We take the max score of the gold answers
      # exact_scores[ids[i]] = max(score, exact_scores.get(ids[i], 0))

      # # True = 1 and False = 0 therefore we count how many True
      # # tp_start += comp_start.count_nonzero()
      # # tp_end += comp_end.count_nonzero()

      # total_cnt += input_ids.size()[0]

      # Calc scores
      for i in range(len(input_ids)):

        total_cnt += 1

        # Check if we have an exact match
        if start_positions[i] == pred_start_positions[i] and end_positions[i] == pred_end_positions[i]:
          # We have an exact match, mark it
          exact_scores[ids[i]] = 1
        else:
          # No match, mark with 0 or keep previous marking
          exact_scores[ids[i]] = max(0, exact_scores.get(ids[i], 0))

        # Get the predicted answer token sequence
        pred_tokens = input_ids[i][pred_start_positions[i]:pred_end_positions[i] + 1].tolist()
        answer_tokens = input_ids[i][start_positions[i]:end_positions[i] + 1].tolist()

        # Check how many predicted tokens correspond to gold answer tokens
        common = collections.Counter(answer_tokens) & collections.Counter(pred_tokens)
        num_same = sum(common.values())

        # If none same, then F1=0 , else use the F1 formula
        if num_same == 0:
          score = 0
        else:
          precision = 1.0 * num_same / len(pred_tokens)
          recall = 1.0 * num_same / len(answer_tokens)
          score = (2 * precision * recall) / (precision + recall)
        
        # We take the max F1 score of the gold answers
        f1_scores[ids[i]] = max(score, f1_scores.get(ids[i], 0))

  f1 = sum(f1_scores.values()) / len(f1_scores)
  em = sum(exact_scores.values()) / len(exact_scores)
  epoch_avg_loss = loss / total_cnt

  return (f1, em, epoch_avg_loss, exact_scores, f1_scores)

In [56]:
# for i in val_loader:
#   batch = i
#   break

In [57]:
f1, em, epoch_avg_loss, _, _ = calc_metrics(model, val_loader)

0it [00:00, ?it/s]

In [58]:
f1, em, epoch_avg_loss

(0.7064151886937955, 0.5786210783392823, tensor(0.4395, device='cuda:0'))

## Calculate F1 score on dataset

In [ ]:
def map_id_to_gold_answers(df):

  # Group by id as validation set has several gold answer
  for id, cols in df.groupby(by='id'):
    # for row in zip(cols.answer_start_heb, cols.answer_end_heb):
      # print(row.answer_start_heb, row.answer_end_heb)
    gold_answers = [x[1].context[x[1].answer_start_heb:x[1].answer_end_heb] for x in cols.iterrows()]
    #gold_answers = [x for x in zip(cols.answer_start_heb, cols.answer_end_heb)]
    print(id, gold_answers)
    if i == 10:
      break
    print()
    i+=1

37. 5727d1953acd2414000ded  ['צחיח למחצה']

38. 5727d1953acd2414000ded38  ['לוטון']

38. 573459cbacc1501500babe38  ["צ'סטרפילד קאונטי"]

56  ['קרדינל', '23:00', '1973']

56 dec8773277331400b4d730  ['פיטר פילז']

56 דצמבר 1673277331400b4d70d  ['']

56 דצמבר 2483277331400b4d711  ["ג'ון מקיין"]

56 דצמבר 2483277331400b4d712  ["רודי ג'וליאני"]

56 דצמבר 2483277331400b4d713  ['הסביבה והכלכלה']

56 דצמבר 4913277331400b4d720  ['שש']

56 דצמבר 4913277331400b4d721  ["לוס אנג'לס טיימס"]
